In [3]:
import numpy as np
import os
from tqdm import tqdm

# plotting 
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import ImageGrid
import PyPDF2

# data manager and analysis
import vodex as vx
import numan as nu

# writing files
import tifffile as tif

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Project structure: 

Provide the project folder with the "processed" folder created in the previous notebook. 

As you keep going with the analysis, the folder will have the following structure: 

```
processed
│   experiment.json <----------------------------------- (DONE in 01) the file that contains everything about the experiment, you are creating it once at the beginning of the processing and reusing ever after
└───dff_movie  <---------------------------------------- (DONE in 01) the dff movie :) 
│   │   dff_movie_0000.tif
│   │   dff_movie_0001.tif
│   │   ... 
└───tscore_volumes  <----------------------------------- (DONE in 02) t-score tif files per pair
│   └───2v3
│       │   tscore_2v3.tif
│   └───3v5
│       │   tscore_3v5.tif
│   └───2v5
│       │   tscore_2v5.tif
│   └───2vB
│       │   tscore_2vB.tif
│   └───3vB
│       │   tscore_3vB.tif
│   └───5vB
│       │   tscore_5vB.tif
│   └───BvB1
│       │   tscore_BvB1.tif
│   └───BvB2
│       │   tscore_BvB2.tif
│   └───BvB3
│       │   tscore_BvB3.tif
└───diff_volumes  <------------------------------------- (DONE in 02) absolute difference tif files per pair
│   └───2v3
│       │   diff_2v3.tif
│   └───3v5
│       │   diff_3v5.tif
│   └───...
└───spots
│   └───imaris  <--------------------------------------- (DONE after 02) ATTENTION : You need to put stuff generated by imaris into this folder!!! 
│       │   └───tscore_2v3_Statistics
│       │       │     tscore_2v3_Position.csv
│       │       │     tscore_2v3_Diameter.csv
│       │       │     ...
│       │   └───tscore_3v5_Statistics
│       │       │     tscore_3v5_Position.csv
│       │       │     tscore_3v5_Diameter.csv
│       │       │     ...
│       │   └───tscore_2v5_Statistics
│       │       │     ...
│       │   └───...
│   └───signals  <-------------------------------------- (DONE in 03, 04 and 05, WILL BE UPDATED in this notebook) json files with the extracted signals, also will have the group info after you added it
│       │   spots_2v3.json
│       │   spots_3v5.json
│       │   spots_2v5.json
│       │     ...
│   └───reports  <---------------------------------- tiffs and pdf with the cells significant in any pairwise comparison
│       └───all_significant  <---------------------- tiffs and pdf with all significant cells per group
│           │   └───signals  <---------------------- (DONE in 05) pdfs with signals
│           │       │     ...
│           │   └───images <------------------------ tif masks 
│           │       │     ...
│       └───groupped  <----------------------------- tiffs and pdf where the cells are groupped based on signal shape .. or anything else you want
│           │   readme.txt  <----------------------- ATTENTION : you need to describe the groups
│           │   └───signals  <---------------------- (DONE in 06) pdfs with signals
│           │       │     ...
│           │   └───images  <----------------------- (DONE in 06) tif masks 
│           │       │     ...
````

# Set project folder

The processed/spots/signals should already exist and have the extracted signals saved in there. 

In [4]:
project_folder = "D:/Code/repos/numan/notebooks/data/2vs3vs5/"
path = os.path.join(project_folder, 'processed')

assert os.path.isdir( os.path.join(path, "spots", "signals")), "the directory 'processed/spots/signals' doesn't exist in the project," \
                                " did you forget to run the previous notebook?"

os.chdir(path)
print(os.getcwd())

experiment = vx.from_json(vx.Experiment,'experiment.json')

D:\Code\repos\numan\notebooks\data\2vs3vs5\processed


# Q1 : Is there an effect of learning and adaptation for early vs late in the trial.

The idea is to look at the first 6, middle 6 and last 6 cycles separately and see if the fish responded better/worse 

In [49]:
# run this cell to get the plotting function 
def plot_split_cycle(spot_tag, experiment, group_tag = None, cells_idx = None):
    
    assert group_tag is None or cells_idx is None, "Can't use both group_tag and cells together"
    
    # prepare the signals
    spots = nu.Spots.from_json(f"spots/signals/spots_{spot_tag}.json")
    
    # initialise the signal plotter
    SLIDING_WINDOW = 15  # in volumes
    significant_signals_dff = spots.signals.as_dff(SLIDING_WINDOW)
    sp = nu.SignalPlotter(significant_signals_dff, experiment)

    # choose what cells to show 
    if group_tag is not None:
        group_title = group_tag
        cells_idx = spots.get_group_idx(spots.groups[group_tag])
    elif cells_idx is not None: 
        group_title = np.array(cells_idx).astype(str)
    else:
        group_title = "all cells"
        cells_idx = np.arange(sp.n_signals)
    
        
    # create titles
    main_titles = [f" DFF; Early, middle and late cycles (6 of each) for {group_title} from {spot_tag} t-score image.\nCell {idx}" for idx in cells_idx]
    
    n_rows = 5
    n_col = 1
    figsize=(10, 12)
    ylabel = ''
    xlabel = ''

    for icell, cell_idx in enumerate(cells_idx):
        fig, axes = plt.subplots(n_rows, n_col, figsize=figsize, dpi=160)
        axes = axes.flatten()
        
        main_title = main_titles[icell]
        fig.suptitle(main_title)
        
        ax = axes[0]
        ax_limits = sp.plot_trace(ax, cell_idx,
                    # front_to_tail will shift the cycleby the set number of voxels
                    # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                    # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                    front_to_tail=3,
                    # what error type to use ( "sem" for SEM or "prc" for 5th - 95th percentile )
                    error_type="sem",
                    # wheather to plot the individual traces
                    plot_individual=True,
                    # the color of the individual traces (if shown)
                    noise_color='-c', 
                    ax_limits = None)

        ax.set_title("All cycles: 1 - 18")
        ax.set_xticklabels([])
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)
        
        ax = axes[1]
        sp.plot_trace(ax, cell_idx,
                    # front_to_tail will shift the cycleby the set number of voxels
                    # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                    # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                    front_to_tail=3,
                    # only plot data from certain cycles
                    cycles = [0,1,2,3,4,5],
                    # what error type to use ( "sem" for SEM or "prc" for 5th - 95th percentile )
                    error_type="sem",
                    # wheather to plot the individual traces
                    plot_individual=False, 
                    mean_color = 'darkorange',
                    ax_limits = ax_limits)
        sp.plot_trace(ax, cell_idx,
                    # front_to_tail will shift the cycleby the set number of voxels
                    # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                    # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                    front_to_tail=3,
                    # only plot data from certain cycles
                    cycles = [6,7,8,9,10,11],
                    # what error type to use ( "sem" for SEM or "prc" for 5th - 95th percentile )
                    error_type="sem",
                    # wheather to plot the individual traces
                    plot_individual=False,
                    mean_color = 'orangered', 
                    ax_limits = ax_limits)
        sp.plot_trace(ax, cell_idx,
                    # front_to_tail will shift the cycleby the set number of voxels
                    # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                    # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                    front_to_tail=3,
                    # only plot data from certain cycles
                    cycles = [12,13,14,15,16,17],
                    # what error type to use ( "sem" for SEM or "prc" for 5th - 95th percentile )
                    error_type="sem",
                    # wheather to plot the individual traces
                    plot_individual=False,
                    mean_color = 'firebrick', 
                    ax_limits = ax_limits)
        ax.legend(['1 - 6', '7 - 12', '13 - 18'],framealpha = 1)
        

        ax.set_title("All cycles' mean and SEM overlayed.")
        ax.set_xticklabels([])
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)

        ax = axes[2]
        sp.plot_trace(ax, cell_idx,
                    # front_to_tail will shift the cycleby the set number of voxels
                    # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                    # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                    front_to_tail=3,
                    # only plot data from certain cycles
                    cycles = [0,1,2,3,4,5],
                    # what error type to use ( "sem" for SEM or "prc" for 5th - 95th percentile )
                    error_type="sem",
                    # wheather to plot the individual traces
                    plot_individual=True,
                    # the color of the individual traces (if shown)
                    noise_color='-c', 
                    ax_limits = ax_limits)

        ax.set_title("Cycles 1 - 6")
        ax.set_xticklabels([])
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)
        
        ax = axes[3]
        sp.plot_trace(ax, cell_idx,
                    # front_to_tail will shift the cycleby the set number of voxels
                    # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                    # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                    front_to_tail=3,
                    # only plot data from certain cycles
                    cycles = [6,7,8,9,10,11],
                    # what error type to use ( "sem" for SEM or "prc" for 5th - 95th percentile )
                    error_type="sem",
                    # wheather to plot the individual traces
                    plot_individual=True,
                    # the color of the individual traces (if shown)
                    noise_color='-c', 
                    ax_limits = ax_limits)

        ax.set_title("Cycles 7 - 12")
        ax.set_xticklabels([])
        ax.set_ylabel(ylabel)
        ax.set_xlabel(xlabel)
        
        ax = axes[4]
        sp.plot_trace(ax, cell_idx,
                    # front_to_tail will shift the cycleby the set number of voxels
                    # so when set to 3, there are 3 blank volumes at the begining and at the end ...
                    # if set to 0, will have 6 leading blanks and will end right after the 5 dots (black bar)
                    front_to_tail=3,
                    # only plot data from certain cycles
                    cycles = [12,13,14,15,16,17],
                    # what error type to use ( "sem" for SEM or "prc" for 5th - 95th percentile )
                    error_type="sem",
                    # wheather to plot the individual traces
                    plot_individual=True,
                    # the color of the individual traces (if shown)
                    noise_color='-c', 
                    ax_limits = ax_limits)

        ax.set_title("Cycles 13 - 18")
        ax.set_xticklabels([])
        ax.set_ylabel(ylabel)
        ax.set_xlabel('Volume in cycle')


    

# Choose cells to plot
You can plot multiple cells per t-score image at once. Please copy the cell to look at different t-score images ( spot_tags)

ATTENTION: Results are not saved anywhere except the notebook 

In [ ]:
spot_tag = "2v3"
cells_idx = [20, 1]
plot_split_cycle(spot_tag, experiment, cells_idx = cells_idx)

# Q2: Anything covaries with the responce? 


# Q3: Cell correlation : Any grups that are Active --> Active; Active --> Inactive ? 

# Q4: % of cells responding to different stimuli 

# Q5: Binarise the responce ? 

# Q6 Optic tectum : retino-tactal map is still in development ... positions, inhibition : some areas start looking " grey" ...etc 

Spacial distribution -- locations of the dots 

Separate experiment: "2" is special --> Orientation of the two matter? like when they are closer to the horisontal plane (looks like eyes of another fish) ? Like social responce? 